In [ ]:
import pandas as pd
import ta
import numpy as np
import time
from binance import Client

In [ ]:
api_key = 'Put your key here'
api_secret = 'Your key is here'
client = Client(api_key,api_secret)

In [ ]:
def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback +'min ago UTC'))
  frame = frame.iloc[:,:6]
  frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  return frame

In [ ]:
df = getminutedata('ADAUSDT', '1m', '100')

In [ ]:
def applytechnicals(df):
  df['%K'] = ta.momentum.stoch(df.High,df.Low,df.Close, window = 14, smooth_window=3)
  df['%D'] = df['%K'].rolling(3).mean()

  df['rsi'] = ta.momentum.rsi(df.Close, window = 14)
  df['macd'] = ta.trend.macd_diff(df.Close)
  df.dropna(inplace=True)


In [ ]:
applytechnicals(df)

In [ ]:
class Signals:

  def __init__(self, df, lags):
    self.df = df
    self.lags = lags

  def gettriger(self):
    dfx = pd.DataFrame()
    for i in range(self.lags + 1):
      mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
      dfx = dfx.append(mask, ignore_index=True)
    return dfx.sum(axis=0)


  def decide(self):
      self.df['trigger'] = np.where(self.gettriger(), 1, 0)
      self.df['Buy'] = np.where((self.df.trigger) &
      (self.df['%K'].between(20,80)) & (self.df['%D'].between(20,80)) &
      (self.df.rsi > 50) & (self.df.macd > 0), 1, 0)





In [ ]:
inst = Signals(df, 30)

In [ ]:
inst.decide()

In [ ]:
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-11-21 19:38:00,0.2975,0.2980,0.2969,0.2978,231110.1,34.246575,28.205603,30.391166,-0.000360,0,0
2022-11-21 19:39:00,0.2979,0.2988,0.2975,0.2976,323785.1,31.506849,31.050228,29.469212,-0.000285,0,0
2022-11-21 19:40:00,0.2976,0.2979,0.2967,0.2968,364215.0,20.547945,28.767123,26.063282,-0.000264,0,0
2022-11-21 19:41:00,0.2968,0.2978,0.2968,0.2972,234970.9,29.687500,27.247432,30.395020,-0.000201,0,0
2022-11-21 19:42:00,0.2972,0.2976,0.2972,0.2972,110711.9,31.147541,27.127662,30.395020,-0.000140,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-21 20:40:00,0.3041,0.3046,0.3039,0.3044,179027.2,46.938776,47.293317,61.018027,0.000089,1,1
2022-11-21 20:41:00,0.3044,0.3049,0.3044,0.3048,52943.3,55.102041,48.013605,63.257191,0.000099,1,1
2022-11-21 20:42:00,0.3048,0.3052,0.3047,0.3049,70918.8,57.142857,53.061224,63.816761,0.000100,1,1


In [ ]:
def strategy(pair, qty, open_postion=False):
  df = getminutedata(pair, '1m', '100')
  applytechnicals(df)
  inst = Signals(df, 25) ## less lag around 3
  inst.decide()
  print(f'current Close is ' +str(df.Close.iloc[-1]))
  if df.Buy.iloc[-1]:
    order = client.create_order(symbol=pair, side='BUY', type="MARKET", quantity=qty)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position=True

  while open_position:
    time.sleep(0.5)
    df = getminutedata(pair, '1m', '2')
    print(f"current Close" +str(df.Close.iloc[-1]))
    print(f"current Target" +str(buyprice *1.005))
    print(f'current Stop is' +str(buyprice*0.995))
    if df.Close[-1] >= buyprice * 0.995 or df.Close[-1] >= 1.005 * buyprice:
        order = client.create_order(symbol=pair, side='SELL', type="MARKET", quantity=qty)
        print(order)
        break


In [1]:
#strategy('ADAUSDT', 50) #DO NOT RUN IT IF YOU"RE NOT SURE

NameError: ignored